In [1]:
import os
import shutil
import nibabel as nib
import numpy as np

mydir = r'D:\20220421-FANBIN\CTOUT'
outdir = r'D:\20220421-FANBIN'

img_dir = os.path.join(outdir, 'images')
msk_dir = os.path.join(outdir, 'masks')
os.makedirs(img_dir, exist_ok=True)
os.makedirs(msk_dir, exist_ok=True)

def get_array(path):
    return np.array(nib.load(path).dataobj)

for i in os.listdir(mydir):
    if os.path.exists(os.path.join(mydir, i, 'PD-COR.nii.gz')) and os.path.exists(os.path.join(mydir, i, 'Talus-label.nii.gz')):
        shutil.copy(os.path.join(mydir, i, 'PD-COR.nii.gz'), os.path.join(img_dir, f"{i}.nii.gz"))
        shutil.copy(os.path.join(mydir, i, 'Talus-label.nii.gz'), os.path.join(msk_dir, f"{i}.nii.gz"))
    elif os.path.exists(os.path.join(mydir, i, 'Segmentation.seg.nii.gz')):
        has = False
        seg = get_array(os.path.join(mydir, i, 'Segmentation.seg.nii.gz'))
        for f in os.listdir(os.path.join(mydir, i)):
            if f != 'Segmentation.seg.nii.gz':
                img = get_array(os.path.join(mydir, i, f))
                if seg.shape == img.shape:
                    has = True
                    shutil.copy(os.path.join(mydir, i, f), os.path.join(img_dir, f"{i}.nii.gz"))
                    shutil.copy(os.path.join(mydir, i, 'Segmentation.seg.nii.gz'), os.path.join(msk_dir, f"{i}.nii.gz"))
                    break
        if not has:
            print(f'新数据：{i}数据存在缺失！')
    else:
        print(f'{i}数据存在缺失！')


1904099735数据存在缺失！
1910221276数据存在缺失！
新数据：1911116371数据存在缺失！
2001209677数据存在缺失！
2005159506数据存在缺失！
新数据：chuping-f-L数据存在缺失！
sunjinzhi-f-坏了数据存在缺失！
新数据：zhaochuanrong-m数据存在缺失！


In [1]:
import os
import shutil
import numpy as np
import SimpleITK as sitk

root = r'D:/20220421-FANBIN/masks'
new_root = os.path.join(os.path.dirname(root), 'masks_12')
os.makedirs(new_root, exist_ok=True)
for m in os.listdir(root):
    origin_img = sitk.ReadImage(os.path.join(root, m))  # 读取原始文件
    
    mask_arry = sitk.GetArrayFromImage(origin_img)
    new_mask = np.zeros_like(mask_arry)
    new_mask[mask_arry == 1] = 1
    new_mask[mask_arry == 3] = 2
    
    origin = origin_img.GetOrigin()  # 这三句是获取的原始图像文件的位置和方向吧。
    spacing = origin_img.GetSpacing()
    direction = origin_img.GetDirection()

    w = sitk.GetImageFromArray(new_mask)

    w.SetOrigin(origin)  # 将自己的文件处理成和官方一致的位置坐标系
    w.SetSpacing(spacing)
    w.SetDirection(direction)
    sitk.WriteImage(w, os.path.join(new_root, m))  # 处理完之后保存到相应的合适位置。